In [1]:
# this could be a class, instantiated with the user's steam key to connect to the api

In [2]:
import pandas as pd
import numpy as np
import requests
import keyring
import time
import re
from IPython.display import clear_output

In [3]:
number_ids_you_want = 100000 # how many unique steam ids do you want? the API queryer will stop when it has more than
                              # this many

In [4]:
password = keyring.get_password(service_name = 'steam', username = 'IcyJoseph')
steam_id = '76561198985648791' # this is my steam_id
new_id = '76561198020258797' # this is an ID I got from reddit - here:
#                             https://www.reddit.com/r/Steam/comments/68yr8k/building_a_big_list_of_steam_ids_hit_me_up_if_you/

In [4]:
def get_player_info(steam_id):
    
    response = requests.get(
        f'http://api.steampowered.com/ISteamUser/GetPlayerSummaries/v0002/?key={password}&steamids={steam_id}')
    return response

In [5]:
def get_friends(steam_id):
    # returns basic profile information
    response = requests.get(
        f'http://api.steampowered.com/ISteamUser/GetFriendList/v0001/?key={password}&steamid={steam_id}&relationship=friend')
    return response

In [30]:
def get_games(steam_id):
    # returns basic profile information
    response = requests.get(
    f'''http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key={password}&steamid={steam_id}
        &format=json&include_appinfo=true''')
    return response

In [ ]:
# me = get_player_info(steam_id)
# friends = get_friends(steam_id)
# games = get_games(steam_id)

In [14]:
# next steps:
    # create function that takes friend's steam_id as an input and returns their top (5? 10?) played games
    # pull some sort of info on players

In [15]:
# takes a steam_id and returns all the id's friends
def find_all_friends(steam_id):
    
    friends = get_friends(steam_id) # query steam api

    splitfriends = str(friends.content).split('"')
    all_friends = [i for i in splitfriends if i.isnumeric() == True] # items which are all numeric are steam_ids
    
    return all_friends

In [20]:
# need to get a list of active steam ids. 
# I'm pulling friends own a known popular ID, then pulling friends of each friend, and so on until I get 1M IDs

counter_start = 0
num_runs = 1
all_ids = list()
all_ids.append(new_id)

while len(all_ids) < number_ids_you_want and counter_start != len(all_ids):
    
    counter_end = counter_start+100 # the steam api can accept up to 100 steam ids at once, so we'll give it 100...
    
    if counter_end > len(all_ids): # ... or as many as we can if we don't have 100
        counter_end = len(all_ids)
        
    id_string = [i+"," for i in all_ids[counter_start:counter_end]] # pull out the ids from our list
    ids = "".join(id_string) # and concatenate them all into one string
        
    time.sleep(0.5)    
        
    steam_ids = find_all_friends(ids)
        
    all_ids.extend(steam_ids) # adds the friends pulled to the list of ids to iterate through, so we can keep pulling friends
    all_ids = set(all_ids)
    all_ids = list(all_ids)
    
    clear_output()
    print(f"this is run number {num_runs}, counter_start is {counter_start}, all_ids is "+str(len(all_ids))+" items long")
    
    counter_start = counter_end
    num_runs += 1

id_df = pd.DataFrame(data = {'id':all_ids}) # export steam ids to csv
id_df.to_csv('all_ids.csv', index = False)

this is run number 5475, all_ids is 547357 items long


### Get Games (work in progress)

In [26]:
first_80k = pd.read_csv('all_ids.csv') # steam will only accept 100k calls a day, and we have 550,000, so have to do this in bits
first_80k = first_80k.iloc[:80000,:]

In [33]:
def users_games(steam_id):
    
    gametext = get_games(steam_id)
    
    # re.finditer returns an iterator object, so we have to use list comprehension to get the index at which the game 
    # names begin
    gamestart = [i.end() for i in re.finditer('name":"',str(newgames.content))]

    gamelist = []

    for start_index in gamestart: # we're pulling out the names of games for each user_id. This is very slow - needs work

        end_index =  re.search('",',str(newgames.content)[start_index:])
        end_index = end_index.start()

        game = str(newgames.content)[start_index:start_index+end_index]

        gamelist.append(game)
        
    return gamelist

In [34]:
newgames = users_games(new_id)

In [35]:
newgames

['Counter-Strike',
 'Day of Defeat',
 'Deathmatch Classic',
 'Ricochet',
 'Counter-Strike: Condition Zero',
 'Counter-Strike: Condition Zero Deleted Scenes',
 'Day of Defeat: Source',
 'Team Fortress Classic',
 'Half-Life: Opposing Force',
 'Half-Life',
 'Half-Life: Blue Shift',
 'Counter-Strike: Source',
 'Half-Life: Source',
 'Half-Life Deathmatch: Source',
 'Red Orchestra: Ostfront 41-45',
 'Mare Nostrum',
 "Darkest Hour: Europe \\'44-\\'45",
 'The Ship',
 'The Ship Single Player',
 'The Ship Tutorial',
 'BloodRayne',
 'BloodRayne 2',
 'Dark Messiah of Might & Magic Single Player',
 'Dark Messiah of Might & Magic Multi-Player',
 "Sid Meier\\'s Pirates!",
 'Prey',
 "Garry\\'s Mod",
 'Call of Duty',
 'Call of Duty 2',
 'Call of Duty: United Offensive',
 'DEFCON',
 'Dreamfall: The Longest Journey',
 'Vampire: The Masquerade - Bloodlines',
 'Joint Task Force',
 'Rogue Trooper',
 'CivCity: Rome',
 'Tomb Raider: Anniversary',
 'Full Spectrum Warrior',
 'Full Spectrum Warrior: Ten Hammers'

In [31]:
players_and_games = pd.DataFrame()

start_row = 0
end_row = 100

id_df.head()